In [3]:
# Load both pre-trained and fine-tuned models and compare the results
import os
from dotenv import load_dotenv
from datasets import load_dataset, DatasetDict, Dataset 
import pandas as pd 
import ast 
import datasets 
from tqdm import tqdm 
import time
import torch

load_dotenv()
access_token = os.getenv("ACCESS_TOKEN")

# Load data set from huggingface 
data_sample = load_dataset("QuyenAnhDE/Diseases_Symptoms")

/Users/royzheng/workArea/workspace/llm-playground/env/lib/python3.10/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [4]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel 

# If you have an NVIDIA GPU attached, use 'cuda' 
if torch.cuda.is_available(): 
    device = torch.device('cuda') 
else: # If Apple Silicon, set to 'mps' - otherwise 'cpu' (not advised) 
    try: 
        device = torch.device('mps') 
    except Exception: 
        device = torch.device('cpu')

# The tokenizer turns texts to numbers (and vice-versa) 
base_tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2') 
# The transformer 
base_model = GPT2LMHeadModel.from_pretrained('distilgpt2').to(device)

In [7]:
# Test base GPT2 model
input_str = "Fetal Alcohol Syndrome" 
input_ids = base_tokenizer.encode(input_str, return_tensors='pt').to(device) 
output = base_model.generate( input_ids, max_length=20, num_return_sequences=1, \
                        do_sample=True, top_k=8, top_p=0.95, temperature=0.5, repetition_penalty=1.2 ) 
decoded_output = base_tokenizer.decode(output[0], skip_special_tokens=True) 
print(decoded_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Fetal Alcohol Syndrome, and the Development of Chronic Fatigue. The authors declare no conflict between


In [9]:
# Test fine-tuned model
#tune_tokenizer = GPT2Tokenizer.from_pretrained('zheng438/distilgpt2-disease-syptom')
# The transformer 
tune_model = GPT2LMHeadModel.from_pretrained('zheng438/distilgpt2-disease-syptom').to(device)

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [16]:
# Test tuned GPT2 model
count = 0
for item in data_sample['train']:
    input_str = item['Name']
    input_ids = base_tokenizer.encode(input_str, return_tensors='pt').to(device) 
    print(f"predicting {count} ...")
    output = tune_model.generate( input_ids, max_length=20, num_return_sequences=1, \
                        do_sample=True, top_k=8, top_p=0.95, temperature=0.5, repetition_penalty=1.2 ) 
    print(f"done {count}.")
    decoded_output = base_tokenizer.decode(output[0], skip_special_tokens=True) 
    if decoded_output != item['Symptoms']:
        print(f"result differs on id={count}, output={decoded_output}, expected={item['Symptoms']}")
    count += 1
   

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


predicting 0 ...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


done 0.
result differs on id=0, output=Panic disorder | Palpitations, Sweating, Trembling, Shortness of breath,, expected=Palpitations, Sweating, Trembling, Shortness of breath, Fear of losing control, Dizziness
predicting 1 ...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

done 395.
result differs on id=395, output=Urinary Stones (Kidney Stones) | Severe abdominal or back pain, blood in urine,, expected=Severe abdominal or back pain, blood in urine, frequent urination, pain during urination
predicting 396 ...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


done 396.
result differs on id=396, output=Osteoporosis | Fragile bones, loss of height over time, back pain, fractures, expected=Fragile bones, loss of height over time, back pain, fractures
predicting 397 ...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


done 397.
result differs on id=397, output=Rheumatoid Arthritis | Joint pain, swelling, stiffness, limited range of motion, fever, expected=Joint pain, stiffness, swelling, fatigue, loss of appetite
predicting 398 ...
done 398.
result differs on id=398, output=Type 1 Diabetes | Frequent urination, Increased thirst, Weight loss, expected=Frequent urination, Increased thirst, Weight loss
predicting 399 ...
done 399.
result differs on id=399, output=Type 2 Diabetes | Frequent urination, Increased thirst, Weight loss, expected=Fatigue, Increased hunger, Slow healing of wounds


In [13]:
# for item in data_sample['train']:
#     print(item['Name'] + ' --  ' + item['Symptoms'])